In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

[fetch_20newsgroups data](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)

In [ ]:
# uzimamo tri teme: racunari, sport i religija
categories = [
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'alt.atheism',
 'soc.religion.christian',
]

In [ ]:
dataset = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

In [ ]:
dataset

In [ ]:
df = pd.DataFrame(dataset.data, columns=["corpus"])

In [ ]:
df.head()

In [ ]:
df.iloc[1].values

In [ ]:
df.shape

### Preporcessing

In [ ]:
# Biblioteka za rad sa tekstom
import re
import string
import nltk
from nltk.corpus import stopwords

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')[:10]

In [ ]:
len(stopwords.words('english'))

In [ ]:
string.punctuation

In [ ]:
def preprocess_text(text):
    # remove special chars (> punctuation) and numbers
    text = re.sub('[^A-Za-z]+', ' ', text)  # ukloni sve sto nije veliko ili malo slovo 
    # remove stopwords
    tokens = nltk.word_tokenize(text) # podela teksta na tokene (kao reci)
    tokens = [w for w in tokens if not w.lower() in stopwords.words('english') and w not in string.punctuation] # izbaci stopwords
    text = ' '.join(tokens) # ocisceni tekst
    text = text.lower().strip() # sve mala slova i bez suvisnih razmaka (whitespace) 
    return text

In [ ]:
df['cleaned_corpus'] = df['corpus'].apply(lambda x: preprocess_text(x))

In [ ]:
df

In [ ]:
df.iloc[0].values

## TF-IDF

<img src = 'tfidf.png'>

**TF-IDF (Term Frequency-Inverse Document Frequency)** je statisticka mera koja govori koliko je svaka rec bitna u korpusu tako sto svakoj reci dodeljuje numericku tezinu. 

In [ ]:
tf_idf_vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.95)

In [ ]:
X = tf_idf_vectorizer.fit_transform(df['cleaned_corpus'])

In [ ]:
X.shape  
# svaka rec dobija svoju tezinu u tekstu, odnosno svakom tekstu dodeljujemo vektor duzine broja reci u vokabularu

In [ ]:
vocabulary = []
for text in df['cleaned_corpus']:
    vocabulary.extend([w for w in nltk.word_tokenize(text)])

In [ ]:
len(np.unique(vocabulary)) # ima malo vise reci jer smo neke filtrirali koristeci max_df

In [ ]:
X # sparse matrix

In [ ]:
X.toarray()

### K-means

In [ ]:
Ks = range(1, 8) 

In [ ]:
models = [KMeans(n_clusters=i, random_state=23) for i in Ks]

In [ ]:
scores = [model.fit(X).score(X) for model in models]

In [ ]:
plt.plot(Ks, scores)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

In [ ]:
clusters = kmeans.labels_ 
clusters

## PCA 

**PCA (Principal Component Analysis)** je tehnika redukcije dimenzije tako da se sto bolje ocuva informacije koju podaci nose. Podaci su predstavljeni glavnim komponentama u novom koordinatnom sistemu dobijenom ortogonalnom transformacijom. Prva glavna komponenta je pravac u prostoru na kome projekcije tacaka podataka imaju najvecu varijansu. U zavisnosti od strukture podataka, mozemo redukovati dimenziju na samo nekoliko glavnih komponenti.

<img src='pca.jpeg' width=600>

In [ ]:
# PCA sa dve glavne komponente
pca = PCA(n_components=2, random_state=42)

In [ ]:
X_pca = pca.fit_transform(X.toarray())
x0 = X_pca[:, 0]
x1 = X_pca[:, 1]

In [ ]:
df['cluster'] = clusters
df['x0'] = x0
df['x1'] = x1

In [ ]:
df.head()

In [ ]:
X_pca.shape

In [ ]:
def get_top_keywords(n_terms):
    df = pd.DataFrame(X.todense()).groupby(clusters).mean()
    terms = tf_idf_vectorizer.get_feature_names_out() # vraca vokabular
    for i, row in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(row)[-n_terms:]])) 

In [ ]:
get_top_keywords(10)  # 10 najvaznijih reci u svakom klasteru

In [ ]:
plt.figure(figsize=(12, 7))
plt.title('PCA rezultat K-means klasterovanja TF-IDF reprezentacije teksta')
plt.xlabel('x0')
plt.ylabel('x1')
sns.scatterplot(data=df, x='x0', y='x1', hue='cluster', palette="viridis")
plt.show()